This notebook automates the entire forecasting workflow - from loading data and preprocessing it, to generating predictions and saving results.
Automation is important because it keeps the process consistent and repeatable, especially when dealing with time-series updates or new data over time. Instead of manually running each step or copying code across notebooks, the pipeline handles everything end-to-end with a single function call.

The results are then saved both as CSV files and into a local SQLite database for easy access and future analysis.

By the end, this setup makes it simple to update forecasts whenever new data arrives, no code rewriting, no manual data handling, just run the pipeline and get updated predictions.

Codes here are from the main `demand_forecasting_ecommerce.ipynb` notebook, it was wrapped in a function to automate the process.

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os
from sqlalchemy import create_engine
from sklearn.preprocessing import StandardScaler
from sqlalchemy import text

In [2]:
# Define preprocessing function
def preprocess_data(df, scaler=None, fit_scaler=False):
    """
      - Convert date to datetime
      - Sort by store,item,date
      - Create time features
      - If `sales` exists: create log_sales, sales_diff (groupwise)
      - Scale `sales` using StandardScaler (fit only if fit_scaler=True)

    Returns:
      df (pd.DataFrame) : transformed dataframe with new columns
      scaler (StandardScaler or None)
    """
    df = df.copy()

    # Ensure datetime
    df["date"] = pd.to_datetime(df["date"])

    # Sort for group operations
    df = df.sort_values(["store", "item", "date"]).reset_index(drop=True)

    # Add time features
    df["year"] = df["date"].dt.year
    df["month"] = df["date"].dt.month
    df["day"] = df["date"].dt.day
    df["dayofweek"] = df["date"].dt.dayofweek
    df["weekofyear"] = df["date"].dt.isocalendar().week.astype(int)
    df["is_weekend"] = (df["dayofweek"] >= 5).astype(int)

    # If sales present, create log and diff features
    if "sales" in df.columns:
        # log transform
        df["log_sales"] = np.log1p(df["sales"])

        # Group-wise differencing
        df["sales_diff"] = df.groupby(["store", "item"])["log_sales"].diff()

        # Scale raw sales for ML models
        if scaler is None:
            scaler = StandardScaler()
        if fit_scaler:
            df["sales_scaled"] = scaler.fit_transform(df[["sales"]])
        else:
            df["sales_scaled"] = scaler.transform(df[["sales"]])
    else:
        scaler = scaler

    return df, scaler

In [3]:
# Create lag and rolling feature
def create_lag_rolling_features(df, lags=[1,7,14,28], roll_windows=[7,14,28], target_col="lag_sales"):
    """
    Add lag_n and rolling_mean_w features based on `target_col`.
    Uses groupby by (store, item) and shift/rolling so no leakage occurs.
    """
    df = df.copy()
    df = df.sort_values(["store", "item", "date"]).reset_index(drop=True)

    # Lags
    for lag in lags:
        df[f"lag_{lag}"] = df.groupby(["store", "item"])[target_col].shift(lag)

    # Rolling means
    for w in roll_windows:
        df[f"rolling_mean_{w}"] = df.groupby(["store", "item"])[target_col] \
                                      .transform(lambda x: x.shift(1).rolling(window=w, min_periods=1).mean())

    return df

In [4]:
# Define X, y builder
def build_X_y(df, feature_cols, target_col="sales"):
    """
    Given a dataframe with features (including lag features), return X, y.
    Drops rows with NaNs in any of the chosen feature_cols or the target (to avoid training on incomplete rows).
    For test (no target_col), returns X and y=None.
    """
    df = df.copy()

    if target_col in df.columns:
        # drop rows where any feature or target is NaN
        keep = df.dropna(subset=feature_cols + [target_col])
        X = keep[feature_cols].reset_index(drop=True)
        y = keep[target_col].reset_index(drop=True)
        return X, y
    else:
        # Test data: drop rows any feature is Nan, return X only
        keep = df.dropna(how="all", subset=feature_cols)
        X = keep[feature_cols].reset_index(drop=True)
        return X, None

In [5]:
# Setup directories for saving outputs
def setup_directories():
    """Create folders for outputs and models."""
    os.makedirs("model_outputs", exist_ok=True)
    os.makedirs("database", exist_ok=True)
    return "database/forecasting.db"

In [6]:
# Load datasets (train, validation, and test)
def load_datasets():
    """Load training, validation, and test data."""
    print("Loading datasets...")
    train_data = pd.read_csv("train_data.csv")
    val_data = pd.read_csv("val_data.csv")
    test = pd.read_csv("test.csv")
    print(f"Train: {train_data.shape} | Val: {val_data.shape} | Test: {test.shape}")
    return train_data, val_data, test

In [7]:
# Preprocess train/val/test datasets with the same preprocessing function
def preprocess_all(train_data, val_data, test):
    """Preprocess all datasets and apply feature engineering."""
    print("Preprocessing data...")
    train_proc, scaler = preprocess_data(train_data, fit_scaler=True)
    val_proc, _ = preprocess_data(val_data, scaler=scaler, fit_scaler=False)
    test_proc, _ = preprocess_data(test, scaler=scaler, fit_scaler=False)

    # Mark dataset origin
    train_proc["__set__"] = "train"
    val_proc["__set__"] = "val"
    test_proc["__set__"] = "test"

    # Combine for feature creation
    combined = pd.concat([train_proc, val_proc, test_proc], axis=0, ignore_index=True)
    combined = combined.sort_values(["store", "item", "date"]).reset_index(drop=True)

    # Create lag and rolling features
    combined = create_lag_rolling_features(combined, target_col="log_sales")

    # Split back into train, validation, and test
    train_lag = combined[combined["__set__"] == "train"].reset_index(drop=True)
    val_lag   = combined[combined["__set__"] == "val"].reset_index(drop=True)
    test_lag  = combined[combined["__set__"] == "test"].reset_index(drop=True)

    # Define feature columns (excluding target 'sales' and 'log_sales')
    features = [
        "store", "item", "year", "month",
        "dayofweek", "is_weekend", "weekofyear",
        "lag_1", "lag_7", "lag_14", "lag_28",
        "rolling_mean_7", "rolling_mean_14", "rolling_mean_28"
    ]

    # Build X, y for training, validation, and test sets
    X_train, y_train = build_X_y(train_lag, feature_cols=features, target_col="sales")
    X_val, y_val = build_X_y(val_lag, feature_cols=features, target_col="sales")
    X_test, _ = build_X_y(test_lag, feature_cols=features, target_col=None)

    # Fill missing values in test data
    X_test = X_test.fillna(X_train.mean())

    print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
    print(f"X_val: {X_val.shape}, y_val: {y_val.shape}")
    print(f"X_test: {X_test.shape}")

    return X_train, y_train, X_val, y_val, X_test, test, train_data, val_data

In [8]:
# Load trained models
def load_models():
    """Load pre-trained models."""
    print("Loading trained models...")
    xgb = joblib.load("xgb_model.pkl")
    lgbm = joblib.load("lgbm_model.pkl")
    rf = joblib.load("rf_model.pkl")
    return xgb, lgbm, rf

In [9]:
# Generate predictions
def generate_predictions(xgb, lgbm, rf, X_test, test):
    """Generate and combine forecasts from models."""
    print("Generating forecasts...")
    xgb_preds = xgb.predict(X_test)
    lgbm_preds = lgbm.predict(X_test)
    rf_preds = rf.predict(X_test)
    ensemble_preds = (0.5 * xgb_preds) + (0.5 * lgbm_preds)

    test_forecast = pd.concat(
        [test.reset_index(drop=True),
         pd.DataFrame({
             "XGB_Preds": xgb_preds,
             "LGBM_Preds": lgbm_preds,
             "RF_Preds": rf_preds,
             "Ensemble": ensemble_preds
         })],
        axis=1
    )
    return test_forecast

In [10]:
# Save results to CSV and database
def save_results(test_forecast, train_data, val_data, db_path):
    """Save the predictions and original data to CSV and SQLite."""
    test_forecast.to_csv("model_outputs/test_forecast_predictions.csv", index=False)
    print("Predictions saved to 'model_outputs/test_forecast_predictions.csv'")

    engine = create_engine(f"sqlite:///{db_path}")
    train_data.to_sql("train_data", engine, index=False, if_exists="replace")
    val_data.to_sql("val_data", engine, index=False, if_exists="replace")
    test_forecast.to_sql("test_predictions", engine, index=False, if_exists="replace")
    print(f"Data stored successfully in {db_path}")

In [11]:
def run_forecasting_pipeline():
    """Run the entire forecasting workflow from start to finish."""
    db_path = setup_directories()
    train_data, val_data, test = load_datasets()
    X_train, y_train, X_val, y_val, X_test, test_df, train_raw, val_raw = preprocess_all(train_data, val_data, test)
    xgb, lgbm, rf = load_models()
    test_forecast = generate_predictions(xgb, lgbm, rf, X_test, test_df)
    save_results(test_forecast, train_raw, val_raw, db_path)
    print("Forecasting pipeline completed successfully.")

In [12]:
# Run the entire pipeline
run_forecasting_pipeline()

Loading datasets...
Train: (730500, 4) | Val: (182500, 4) | Test: (45000, 4)
Preprocessing data...
X_train: (716500, 14), y_train: (716500,)
X_val: (182500, 14), y_val: (182500,)
X_test: (45000, 14)
Loading trained models...
Generating forecasts...
Predictions saved to 'model_outputs/test_forecast_predictions.csv'
Data stored successfully in database/forecasting.db
Forecasting pipeline completed successfully.


**CLEAN DATA TYPES FOR POWER BI**

In [13]:
# Create power bi friendly table
db_path = "database/forecasting.db"
engine = create_engine(f"sqlite:///{db_path}")
with engine.connect() as conn:
    conn.execute(text("DROP VIEW IF EXISTS forecast_powerbi_view;"))
    conn.execute(text("DROP TABLE IF EXISTS forecast_clean;"))

    conn.execute(text("""
          CREATE TABLE forecast_clean AS
        SELECT
            CAST(id AS INTEGER) AS id,
            CAST(date AS TEXT) AS date,
            CAST(store AS INTEGER) AS store,
            CAST(item AS INTEGER) AS item,
            CAST(XGB_Preds AS FLOAT) AS XGB_Preds,
            CAST(LGBM_Preds AS FLOAT) AS LGBM_Preds,
            CAST(RF_Preds AS FLOAT) AS RF_Preds,
            CAST(Ensemble AS FLOAT) AS Ensemble
        FROM test_predictions;
    """))

print("forecast_clean table created successfully for Power BI")

forecast_clean table created successfully for Power BI
